In [2]:
import torch
from torch import nn
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
from helpers.plot import plot_fit
import matplotlib.pyplot as plt
from lightning_trainer import UnetDACLighting
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.loggers import WandbLogger, TensorBoardLogger

from audio_dataset import DictTorchPartedDataset, PinDictTorchPartedDataset
from typing import List
from trainer import AudioTrainer
import os
import pandas as pd

from unet_dac import UnetDAC
import lightning as L

In [3]:
from config import NUM_MICS, ANGLE_RES

L_v = 96
K = 256
# INPUT_LEN = 64
# VIRTUAL_BATCH_SIZE = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UnetDAC(L=L_v, K=K, M=NUM_MICS).to(device)

In [4]:
lr = 1e-3

train_bs = 16
validation_bs = train_bs

model_name = f"unet_doa_batch{train_bs}_lr{lr:.0e}"
train_dataset = PinDictTorchPartedDataset('data_batches', 'train06r076' , ['samples', 'target'], real_batch_size=64, virtual_batch_size=1, device=device)
train_dataloader = DataLoader(train_dataset, batch_size=train_bs, shuffle=True, num_workers=4, persistent_workers=True, prefetch_factor=16)

validation_dataset = PinDictTorchPartedDataset('data_batches', 'validation06r076' , ['samples', 'target'], real_batch_size=64, virtual_batch_size=1, device=device)
valiadtion_dataloader = DataLoader(validation_dataset, batch_size=validation_bs, shuffle=True, num_workers=4, persistent_workers=True, prefetch_factor=16)

test_dataset = PinDictTorchPartedDataset('data_batches', 'test10r0235revrad' , ['samples', 'ref_stft', 'target', 'mixed_signals'], real_batch_size=30, virtual_batch_size=1, device=device)
test_dataloader = DataLoader(test_dataset, batch_size=30, shuffle=False)

criterion = nn.CrossEntropyLoss()
model_lighting = UnetDACLighting(model, criterion, lr)
# wandb_logger = WandbLogger(log_model="all", project='AudioDOA', name='bs=64,sig0.6 clean. 0.76 with reverb')
logger = TensorBoardLogger("tb_logs", name=model_name)

trainer = L.Trainer(max_epochs=100,
                    callbacks=[EarlyStopping(monitor="val_loss", mode="min", patience=3)],
                    default_root_dir=model_name,
                    log_every_n_steps=9,
                    logger=logger)
trainer.fit(model_lighting, train_dataloaders=train_dataloader, validation_dataloaders=valiadtion_dataloader, test_dataloaders=test_dataloader)

KeyError: 'ref_stft'

In [ ]:
d1['probs'] = torch.tensor(outputs.cpu().detach().numpy())
print(d1.keys())
# for k,v in d1.items():
    # d1[k] = torch.tensor(v.cpu().detach().numpy())
torch.save(d1, 'example_batch2.pt')


dict_keys(['samples', 'ref_stft', 'target', 'perceived_signals', 'doas', 'probs'])
